--Importación de librerias--

In [ ]:
#Importando librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import load_model

#sklearn.processing para normalizar los datos
from sklearn.preprocessing import MinMaxScaler

# Importando el modelo LSTM de Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Importando la función EarlyStopping para evitar el sobreajuste
from tensorflow.keras.callbacks import EarlyStopping

--Definiendo la ruta y asignando el Dataset--

In [ ]:
#Cargamos el dataset de temperaturas mínimas diarias en Melbourne
#URL del dataset: https://www.kaggle.com/datasets/paulbrabban/daily-minimum-temperatures-in-melbourne
melb_temp_path = "D:\\Hacking\\Python\\AI_Learning\\Aprendizaje_Profundo\\Temperature_Melb_LSTM\\daily-minimum-temperatures-melb.csv"
melb_temp_data = pd.read_csv(melb_temp_path)

--Comprensión de la estructura del Dataset--

In [ ]:
print(melb_temp_data.head())
print(melb_temp_data.info())

--Tratamiento y ajuste de las columnas del Dataset--

In [ ]:
#Renombramos la columna 'Dayly minimum temperature in Melbourne, Australia' a 'Temp'
#inplace=True permite modificar el DataFrame original sin crear una copia
melb_temp_data.rename(columns={'Daily minimum temperatures in Melbourne, Australia, 1981-1990': 'Temp'}, inplace=True)

#Convertimos la columna 'Date' a tipo datetime
melb_temp_data['Date'] = pd.to_datetime(melb_temp_data['Date'])
#Establecemos la columna 'Date' como índice del DataFrame
melb_temp_data.set_index('Date', inplace=True)

#Limpiamos los datos de temperatura, convirtiéndolos a numérico y eliminando los valores NaN
melb_temp_data['Temp'] = pd.to_numeric(melb_temp_data['Temp'], errors='coerce')
melb_temp_data.dropna(subset=['Temp'], inplace=True)

--Preparación de los datos--

In [ ]:
#Definimos el escalador MinMaxScaler para normalizar los datos
#feature_range=(0, 1) normaliza los datos entre 0 y 1
scaler = MinMaxScaler(feature_range=(0, 1))
#Ajustamos el escalador a los datos de temperatura
temp_scaled = scaler.fit_transform(melb_temp_data[['Temp']])

In [ ]:
#Función para crear secuencias temporales
#X es la secuencia de entrada 
#y es el valor a predecir
def create_sequences_multistep(data, input_steps, output_steps):
    X, y = [], []
    for i in range(len(data) - input_steps - output_steps):
        X.append(data[i:(i + input_steps), 0])
        y.append(data[i + input_steps:i + input_steps + output_steps, 0])
    return np.array(X), np.array(y)

#Tamaño de la ventana de tiempo
steps = 30
output_steps = 1  # Número de días a predecir
#Creamos las secuencias temporales
X, y = create_sequences_multistep(temp_scaled, steps, output_steps)

--Entrenamiento del modelo--

In [ ]:
#Dividir los datos en conjuntos de entrenamiento y prueba
#No usamos train_test_split para mantener la secuencia temporal
train_size = int(len(X) * 0.7) # 70% para entrenamiento
val_size = int(len(X) * 0.1)  # 10% para validación

train_X = X[:train_size]
train_y = y[:train_size]

val_X = X[train_size:train_size + val_size]
val_y = y[train_size:train_size + val_size]

test_X = X[train_size + val_size:]
test_y = y[train_size + val_size:]

#Creacion del modelo LSTM
model = Sequential([
    
    #Definimos la forma de entrada del modelo
    Input(shape=(steps, 1)),
    #Primera capa LSTM con 64 unidades, activación ReLU y forma de entrada definida por 'steps'
    #La activación 'tanh' es común en RNNs para manejar la no linealidad [Rango de salida: -1 a 1]
    LSTM(64, activation='tanh', return_sequences=True),
    #Dropout para evitar el sobreajuste con una tasa de 0.2
    Dropout(0.2),
    
    #Segunda capa LSTM con 32 unidades y activación 'tanh'
    LSTM(32, activation='tanh'),
    #Otra capa Dropout para evitar el sobreajuste
    Dropout(0.2),
    
    #Dense para la capa de salida con una sola unidad
    Dense(output_steps) #Salida
])

#Definimos EarlyStopping para detener el entrenamiento si no hay mejora en la validación
#patience=10 significa que se detendrá si no hay mejora durante 10 epochs
early_stopping = EarlyStopping(
    patience=10,  # Número de épocas sin mejora antes de detener el entrenamiento
    restore_best_weights=True,  # Restaurar los mejores pesos al final del entrenamiento
)
#Compilación del modelo
#Usamos el optimizador Adam y la función de pérdida de error cuadrático medio (mean_squared_error)
model.compile(optimizer='adam', loss='mean_squared_error')
#Resumen del modelo
model.summary()

In [ ]:
#Entrenamiento del modelo
#El modelo LSTM espera entradas con la forma (n_samples, steps, 1)
model.fit(
    train_X.reshape(-1, steps, 1), #Reshape de train_X para que tenga la forma optima (n_samples, steps, 1)
    train_y,
    validation_data=(val_X.reshape(-1, steps, 1), val_y),  #Reshape de val_X para la validación
    epochs=200,  #Aumentamos el número de epochs a 200 para un entrenamiento más prolongado
    batch_size=32, #El tamaño del batch es 32
    callbacks=[early_stopping]  #Usamos EarlyStopping para evitar el sobreajuste
) 


--Evaluación del modelo--

In [ ]:
#Predicción del modelo
pred_y = model.predict(test_X.reshape(-1, steps, 1))

#Inversión de la normalización para obtener los valores originales
pred_y_inv = scaler.inverse_transform(pred_y.reshape(-1, 1)).reshape(pred_y.shape)
test_y_inv = scaler.inverse_transform(test_y.reshape(-1, 1)).reshape(test_y.shape)

#Cálculo de las métricas de evaluación
rmse = np.sqrt(mean_squared_error(test_y_inv, pred_y_inv))
mae = mean_absolute_error(test_y_inv, pred_y_inv)
r2 = r2_score(test_y_inv, pred_y_inv)
mape = np.mean(np.abs((test_y_inv - pred_y_inv) / test_y_inv)) * 100


#Mostramos los resultados
print("Resultados del modelo LSTM:")
print(f"Error promedio de ~{mae:.1f}ºC a {rmse:.1f}ºC")
print(f"Variabilidad del: {r2 * 100:.2f}%")
print(f"Porcentaje de error medio: {mape:.2f}%")


--Grafico de predicciones frente a valores reales--

In [ ]:
#Visualización de los resultados a través de un gráfico
plt.figure(figsize=(12, 5))
plt.plot(test_y_inv, label='Valores Reales', color='blue')
plt.plot(pred_y_inv, label='Predicciones', color='red')
plt.title('Predicción de Temperatura con LSTM')
plt.xlabel('Días')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.show()

In [ ]:
#Grafico con el error de predicción diario
abs_error = np.abs(test_y_inv - pred_y_inv)
plt.figure(figsize=(12, 4))
plt.plot(abs_error, label='Error Absoluto Diario', color='orange')
plt.title('Error Absoluto Diario de Predicción de Temperatura')
plt.xlabel('Días')
plt.ylabel('Error Absoluto (°C)')
plt.legend()
plt.show()

--Guardado del modelo entrenado--

In [ ]:
#Guardado del modelo entrenado
model.save('D:\\Hacking\\Python\\AI_Learning\\Aprendizaje_Profundo\\Temperature_Melb_LSTM\\melb_temp_LSTM_model.keras')

#Guardado del escalador para normalizar los datos
joblib.dump(scaler, 'D:\\Hacking\\Python\\AI_Learning\\Aprendizaje_Profundo\\Temperature_Melb_LSTM\\melb_temp_scaler.pkl')


--Cargado el modelo entrenado--

In [ ]:
#Cargado del escalador para normalizar los datos
scaler = joblib.load('D:\\Hacking\\Python\\AI_Learning\\Aprendizaje_Profundo\\Temperature_Melb_LSTM\\melb_temp_scaler.pkl')

model_loaded = load_model('D:\\Hacking\\Python\\AI_Learning\\Aprendizaje_Profundo\\Temperature_Melb_LSTM\\melb_temp_LSTM_model.keras')

--Predicción sobre los próximos 7 dias--

In [ ]:
#Tomamos la última secuencia de datos normalizados
last_seq = temp_scaled[-steps:]  # Última secuencia de datos normalizados
last_seq = last_seq.reshape(1, steps, 1)  # Reshape para el modelo LSTM

#Predicción de los próximos 7 días de forma directa (seq2seq)
future_preds = model_loaded.predict(last_seq)
future_preds_inv = scaler.inverse_transform(future_preds.reshape(-1, 1))

print("Predicciones para los próximos 7 días:")
#Bucle para imprimir las predicciones
for i, temp in enumerate(future_preds_inv.flatten(), start=1):
    print(f"Día {i}: {temp:.2f}°C")

In [ ]:
#Generación de fechas para las predicciones futuras
future_dates = pd.date_range(start=melb_temp_data.index[-1] + pd.Timedelta(days=1), periods=output_steps)

#Visualización de las predicciones para los próximos 7 días
plt.figure(figsize=(10, 5))
plt.plot(melb_temp_data.index[-30:], scaler.inverse_transform(temp_scaled[-30:]), label='Últimos 30 dias', color='blue')
plt.plot(future_dates, future_preds_inv, label='Predicción próximos 7 dias', marker='o', color='red')
plt.title('Predicción de Temperatura para los Próximos 7 Días')
plt.xlabel('Fecha (Día/Mes)')
plt.ylabel('Temperatura (°C)')
plt.legend()

#Formateo de las fechas en el eje x
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%d/%m'))
plt.gcf().autofmt_xdate()  # Rotar las etiquetas del eje x para mejor visibilidad
plt.show()